In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.feature_selection \
        import mutual_info_regression
        
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [3]:
## Top 500 dataset

DATA_STORE = Path('/home/sayem/Desktop/Project/data/dataset.h5')

lock_path = "/tmp/assets_h5_file.lock"  # Choose a path for the lock file

from filelock import FileLock

with FileLock(lock_path):
    with pd.HDFStore(DATA_STORE) as store:
        print(sorted(store.keys()))

['/data_YEAR_2013_01_02_to_2014_12_08', '/data_YEAR_2014_12_09_to_2016_11_14', '/data_YEAR_2016_11_15_to_2018_10_22', '/data_YEAR_2018_10_23_to_2020_09_28', '/data_YEAR_2020_09_29_to_2022_09_02', '/data_YEAR_2022_09_06_to_2023_08_11']


In [4]:
import dask.dataframe as dd
import pandas as pd

FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"

# Extract all keys from the HDF5 file
with pd.HDFStore(FILE_PATH) as store:
    available_keys = [key for key in store.keys() if key.startswith('/data_YEAR_')]

# Dictionary to store datasets corresponding to each key
datasets = {}

for FILE_NAME in available_keys:
    # Read the dataset using Dask for each key
    data = dd.read_hdf(FILE_PATH, FILE_NAME)
    # Compute the result (this will load data into memory) 
    # and store it in the datasets dictionary with the key as the identifier
    result = data.compute()
    break

# Now, the datasets dictionary contains data for each key.


In [5]:
result

feature_open  feature_high  feature_low  feature_close  \
ticker date                                                                 
AA     2013-01-03     18.904100     19.306499    18.883200      19.137600   
       2013-01-04     19.222000     19.579800    19.200300      19.537100   
       2013-01-07     19.537100     19.600599    19.200300      19.200300   
       2013-01-08     19.327400     19.389099    19.158400      19.200300   
       2013-01-09     19.685101     19.685101    19.115700      19.158400   
...                         ...           ...          ...            ...   
ZTS    2014-12-02     42.057301     42.607101    41.969601      42.406898   
       2014-12-03     42.406898     42.664799    42.075199      42.538399   
       2014-12-04     42.451698     42.558300    41.561298      41.706699   
       2014-12-05     41.677799     42.139000    41.542301      41.687698   
       2014-12-08     41.893002     42.075199    41.581100      41.801300   

                   feature_volume  feature_market_cap  feature_sector  \
ticker date                                                             
AA     2013-01-03    1.057118e+07        5.703249e+09             4.0   
       2013-01-04    8.690739e+06        5.703249e+09             4.0   
       2013-01-07    1.031992e+07        5.703249e+09             4.0   
       2013-01-08    1.609879e+07        5.703249e+09             4.0   
       2013-01-09    1.979932e+07        5.703249e+09             4.0   
...                           ...                 ...             ...   
ZTS    2014-12-02    2.638230e+06        8.818486e+10             2.0   
       2014-12-03    2.118850e+06        8.818486e+10             2.0   
       2014-12-04    4.531011e+06        8.818486e+10             2.0   
       2014-12-05    3.496368e+06        8.818486e+10             2.0   
       2014-12-08    6.213503e+06        8.818486e+10             2.0   

                   feature_ret_frac_order  feature_ret_01d  feature_ret_02d  \
ticker date                                                                   
AA     2013-01-03                8.914233         0.020875         0.003276   
       2013-01-04                8.914233         0.020875         0.003276   
       2013-01-07                8.914233        -0.017239         0.003276   
       2013-01-08                8.914233         0.000000        -0.017239   
       2013-01-09                8.914233        -0.002182        -0.002182   
...                                   ...              ...              ...   
ZTS    2014-12-02                5.321323         0.008312        -0.004234   
       2014-12-03                5.295430         0.003101         0.011439   
       2014-12-04                4.335114        -0.019552        -0.016511   
       2014-12-05                4.550928        -0.000456        -0.019998   
       2014-12-08                4.708047         0.002725         0.002268   

                   feature_ret_03d  feature_ret_04d  feature_ret_05d  \
ticker date                                                            
AA     2013-01-03         0.003276         0.001087        -0.011062   
       2013-01-04         0.003276         0.001087        -0.011062   
       2013-01-07         0.003276         0.001087        -0.011062   
       2013-01-08         0.003276         0.001087        -0.011062   
       2013-01-09        -0.019384         0.001087        -0.011062   
...                            ...              ...              ...   
ZTS    2014-12-02         0.001578         0.018904         0.003654   
       2014-12-03        -0.001146         0.004684         0.022064   
       2014-12-04        -0.008336        -0.020675        -0.014960   
       2014-12-05        -0.016960        -0.008788        -0.021121   
       2014-12-08        -0.017328        -0.014281        -0.006087   

                   feature_ret_10d  feature_ret_21d  feature_ret_42d  \
ticker date                   

In [6]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from utils import optimize_dataframe
from numba import jit

@jit(nopython=True)
def compute_correlation(data1, data2):
    n = len(data1)
    mean_x = np.mean(data1)
    mean_y = np.mean(data2)
    
    num = np.sum((data1 - mean_x) * (data2 - mean_y))
    den = np.sqrt(np.sum((data1 - mean_x)**2) * np.sum((data2 - mean_y)**2))
    
    if den == 0:
        return np.nan
    else:
        return num / den

def calculate_ic(dataframe, target_column, target_ranked, n_jobs=-1):
    # Exclude the target column from the feature list
    features = [col for col in dataframe.columns.tolist() if col != target_column]
    correlations = Parallel(n_jobs=n_jobs)(
        delayed(compute_correlation)(dataframe[column].values, target_ranked) for column in features
    )
    ic_original = pd.Series(dict(zip(features, correlations))).sort_values(ascending=False)
    return ic_original

def calculate_ic_batched(dataframe, target_column, batch_size=50, corr_threshold=0.5, n_jobs=-1):
    df_ranked = dataframe.rank()
    target_ranked = df_ranked[target_column].values
    columns = [col for col in dataframe.columns.tolist() if col != target_column]
    

    ic_aggregated = pd.Series(dtype=float)

    num_batches = len(columns) // batch_size + 1
    for i in range(num_batches):
        print(f"Processing batch {i+1}/{num_batches}...")
        start_col = i * batch_size
        end_col = start_col + batch_size

        subset_cols = columns[start_col:end_col]
        subset = df_ranked[subset_cols]
        # ic_original = calculate_ic(subset, target_ranked, n_jobs=n_jobs)
        ic_original = calculate_ic(subset, target_column, target_ranked, n_jobs=n_jobs)
        ic_aggregated = ic_aggregated.add(ic_original, fill_value=0)

    ic_aggregated = ic_aggregated.sort_values(ascending=False)

    correlation_matrix = df_ranked[ic_aggregated.index].corr()
    dropped_features = set()
    for col in ic_aggregated.index:
        if col not in dropped_features:
            correlated_features = correlation_matrix[col][(correlation_matrix[col].abs() \
                > corr_threshold) & (correlation_matrix[col].index != col)].index
            for feature in correlated_features:
                if ic_aggregated[col] < ic_aggregated[feature]:
                    dropped_features.add(col)
                else:
                    dropped_features.add(feature)

    ic_reduced = ic_aggregated.drop(labels=dropped_features)
    columns_to_include = ic_reduced.index.tolist() + [target_column, \
        'OPEN', 'HIGH', 'LOW', 'VOLUME', 'CLOSE']
    reduced_dataframe = dataframe[columns_to_include]

    selected_correlation_matrix = \
        correlation_matrix.loc[ic_reduced.index, ic_reduced.index]
    
    return reduced_dataframe, ic_reduced, selected_correlation_matrix


    # return reduced_dataframe

# Optimize memory and clean dataframe
data = optimize_dataframe(result)

# Drop duplicated rows from the data
data.drop_duplicates(inplace=True)

# Drop duplicated columns
data = data.loc[:, ~data.columns.duplicated()]

del result

TARGET = 'RET_FWD_FRAC_ORDER'
reduced_dataframe, selected_ics, selected_corr_matrix = calculate_ic_batched(data, TARGET, batch_size=100)

# Find duplicate columns
duplicated_cols = reduced_dataframe.columns[reduced_dataframe.columns.duplicated()].to_list()

# Drop duplicate columns
reduced_dataframe = reduced_dataframe.loc[:, ~reduced_dataframe.columns.duplicated()]

print(f"Removed duplicate columns: {duplicated_cols}")
print(reduced_dataframe.shape)

del data

Data memory before optimization: 1824.21 MB
Data memory after optimization: 1799.00 MB
Reduced by: 1.38%


KeyError: 'RET_FWD_FRAC_ORDER'

In [ ]:
from utils import save_to_hdf

FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"
# Define key name

FILE_NAME_PREFIX = f'/data/ic_based_reduced_features_YEAR'

_ = save_to_hdf(reduced_dataframe, FILE_PATH, FILE_NAME_PREFIX)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import seaborn as sns

# Assuming selected_corr_matrix is the variable containing your correlation matrix
# selected_corr_matrix = your_correlation_matrix_here

# Define the color map
cmap = sns.diverging_palette(10, 220, as_cmap=True)

# Create the clustermap
g = sns.clustermap(selected_corr_matrix, cmap=cmap, figsize=(30, 30))

plt.show()

In [ ]:
# Sort the Series for better visualization
sorted_ics = selected_ics.sort_values()

# Create the plot
plt.figure(figsize=(10, 20))  # Adjust the size as needed

sns.barplot(x=sorted_ics, y=sorted_ics.index, palette="coolwarm")

plt.xlabel('Information Coefficient (IC)')
plt.ylabel('Features')
plt.title('Information Coefficient (IC) of Selected Features')

# Annotate each bar with the actual IC value
for index, value in enumerate(sorted_ics):
    plt.text(value, index, f'{value:.2f}')

plt.tight_layout()

plt.show()

In [ ]:
from utils import CustomBackwardMultipleTimeSeriesCV

cv_splits = CustomBackwardMultipleTimeSeriesCV(reduced_dataframe, \
        train_period_length=5, test_period_length=3, lookahead=1)

for train_idx, test_idx in cv_splits:
    # Here you can use train_idx and test_idx to get the training and validation data respectively
    train_data = reduced_dataframe[train_idx]
    print(train_data.shape)
    # val_data = reduced_dataframe[test_idx]
    # # Your training and validation code goes here...
    break

In [ ]:
import matplotlib.pyplot as plt

def plot_time_series_cv_splits(dataframe, train_period_length=21*3, \
    test_period_length=21, lookahead=1, date_idx='date'):
    """
    Plots the train and validation windows for time series cross-validation.
    
    Parameters:
        dataframe: The data to be split.
        train_period_length: The number of business days in the training set.
        test_period_length: The number of business days in the validation set.
        lookahead: The gap between training and validation sets.
        date_idx: The name of the date index in the dataframe.
    """
    
    cv_splits = CustomBackwardMultipleTimeSeriesCV(dataframe, \
        train_period_length, test_period_length, lookahead, date_idx)
    
    fig, ax = plt.subplots(figsize=(15, 5))
    
    for i, (train_idx, test_idx) in enumerate(cv_splits):
        # Get the date ranges
        train_dates = dataframe.index.get_level_values(date_idx)[train_idx]
        test_dates = dataframe.index.get_level_values(date_idx)[test_idx]

        ax.plot([train_dates[0], train_dates[-1]], [i, i], color='blue', label='Training' if i == 0 else "")
        ax.plot([test_dates[0], test_dates[-1]], [i, i], color='red', label='Validation' if i == 0 else "")
        ax.scatter([train_dates[0], train_dates[-1]], [i, i], color='blue')
        ax.scatter([test_dates[0], test_dates[-1]], [i, i], color='red')
        
        # Draw vertical dashed lines to represent the lookahead gap for every split
        gap_start = train_dates[-1]
        gap_end = test_dates[0]
        ax.axvline(x=gap_start, color='green', linestyle='--', alpha=0.6)
        ax.axvline(x=gap_end, color='green', linestyle='--', alpha=0.6)

        # Shade the lookahead gap
        ax.axvspan(gap_start, gap_end, facecolor='gray', alpha=0.2)
        
    ax.set_xlabel('Date')
    ax.set_ylabel('Iteration')
    ax.set_title('Backward-Rolling Time Series CV Splits with Lookahead Gap')
    ax.legend(loc='best')
    plt.gca().invert_yaxis()  # invert y-axis to have the most recent data at the top
    plt.show()

# Now call the function to plot
plot_time_series_cv_splits(reduced_dataframe)